In [1]:
import itertools
import pandas as pd
import numpy as np
import random
import csv
import time

import matplotlib.pyplot as plt

import tensorflow as tf

import keras.backend as K
from keras import Sequential
from keras.layers import Dense, Dropout

## DataGenerator
---

In [2]:
class DataGenerator():
    def __init__(self, datapath):
        ''' 
        Load data from the DB Books
        List the users and items
        List all the users historic
        '''
        self.data = self.load_datas(datapath)
        self.users = self.data['user'].unique()
        self.items = self.data['item'].unique()
        self.histo = self.gen_histo()
        self.train = []
        self.test = []
        

    def load_datas(self, datapath):
        '''
        Load the data and merge the name of each books
        A row corresponds to a rate given by a user to books

        Parameters
        ----------
        datapath:   string, path to the data books contain user, item, rating, timestamp

        Returns
        -------
        result:     DataFram, contains all the ratings
        '''
        data = pd.read_csv(datapath, names=['item', 'user', 'rating', 'timestamp'])
        data = data[:1000]
        print(data)

        return data
    
    def gen_histo(self):
        '''
        Group all rates given by users and store them from older to most recent

        Returns
        -------
        result:     List(DataFrame), List of the historic for each user
        '''
        historic_user = []
        for i, u in enumerate(self.users):
            temp = self.data[self.data['user'] == u]
            temp = temp.sort_values('timestamp').reset_index()
            temp.drop('index', axis=1, inplace=True)
            historic_user.append(temp)
        return historic_user
    
    def sample_histo(self, user_histo, action_ratio=0.8, 
                     max_samp_by_user=5, max_state=100, max_action=50, nb_states=[], nb_actions=[]):
        '''
        action이 이 코드에 따르면 각 아이템에 대한 rating이 action인 것으로 보이는데 book이어야 하는 것아닌가?
        '''
        '''
        For a given historic, make one or multiple sampling.
        If no optional argument given for nb_states and nb_actions, 
        then the sampling is random and each sample can have differents size for action and state.
        To normalize sampling we need to give list of the numbers of states and actions to be sampled

        Parameters
        ----------
        user_histo:         DataFrame, historic of user
        delimiter:          float, optional delimiter for the csv
        action_ratio:       float, optional ratio form which books in history will be selected
        max_samp_by_user:   int, optional Number max of sample to make by user
        max_state:          int, optional Number max of books to take for the 'state' column
        max_action:         int, optional Number max of books to take for the 'action' column
        nb_state:           array(int), optional Numbers of books to be taken for each sample made on user's historic
        nb_actions:         array(int), optional Numbers of rating to be taken for each sample made on user's historic

        Returns
        -------
        states:             List(String), All the states sampled, format of a sample: item & rating
        actions:            List(String), All the actions sampled, format of a sample: item & rating

        Notes
        -------
        States must be before(timestamp) the actions.
        If given, size of nb_states is the number of sample by user size of nb_states and nb_actions must be equals
        '''

        n = len(user_histo)
        print(n)
        sep = int(action_ratio * n)
        nb_sample = random.randint(1, max_samp_by_user)
        if not nb_states:
            nb_states = [min(random.randint(1, sep), max_state) for i in range(nb_sample)]
        if not nb_actions:
            nb_actions = [min(random.randint(1, n-sep), max_action) for i in range(nb_sample)]
        
        assert len(nb_states) == len(nb_actions)

        states = []
        actions = []

        # SELECT SAMPLES IN HISTO
        for i in range(len(nb_states)):
            sample_states = user_histo.iloc[0:sep].sample(nb_states[i])
            sample_actions = user_histo.iloc[-(n-sep):].sample(nb_actions[i])

            sample_state = []
            sample_action = []
            for j in range(nb_states[i]):
                row = sample_states.iloc[j]
                # FORMAT STATE
                state = str(row.loc['item']) + '&' + str(row.loc['rating'])
                sample_state.append(state)
            
            for j in range(nb_actions[i]):
                row = sample_actions.iloc[j]
                # FORMAT ACTION
                action = str(row.loc['item']) + '&' + str(row.loc['rating'])
                sample_action.append(action)
            
            states.append(sample_state)
            actions.append(sample_action)
        
        return states, actions

    def gen_train_test(self, test_ratio, seed=None):
        '''
        Shuffle the historic of users and seperate it in a train and a test set.
        Store the ids for each set.
        An user can't be in both set.

        Parameters
        -----------
        test_ratio:     float, ratio to control the sizes of the sets
        seed:           float, seed on the shuffle
        '''

        n = len(self.histo)

        if seed is not None:
            random.Random(seed).shuffle(self.histo)
        else:
            random.shuffle(self.histo)

        self.train = self.histo[:int((test_ratio * n))]
        self.test = self.histo[int((test_ratio * n)):]
        self.user_train = [h.iloc[0,0] for h in self.train]
        print(self.user_train)
        self.user_test = [h.iloc[0,0] for h in self.test]

    def write_csv(self, filename, histo_to_write, delimiter=';', action_ratio=0.8, 
                  max_samp_by_user=5, max_state=100, max_action=50, nb_states=[], nb_actions=[]):
        '''
        From a given historic, create a csv file with the format
        Columns:        state, action_reward, n_state
        Rows:           item&rating1 | item&rating2 | ...item&rating3 |... at filename location.

        Paramters
        ----------
        filename:           string, path to the file to be produced
        histo_to_write:     list(DataFrame), list of the historic for each user
        delimiter:          string, optional delimiter for the csv
        action_ratio:       float, optional ratio form which books in history will be selected
        max_samp_by_user:   int, optional Number max of sample to make by user
        max_state :         int, optional Number max of books to take for the 'state' column
        max_action :        int, optional Number max of books to take for the 'action' action
        nb_states :         array(int), optional Numbers of books to be taken for each sample made on user's historic
        nb_actions :        array(int), optional Numbers of rating to be taken for each sample made on user's historic

        Notes
        -----
        if given, size of nb_states is the number of sample by user sizes of nb_states and nb_actions must be equals
        '''
        with open(filename, mode='w') as file:
            f_writer = csv.writer(file, delimiter=delimiter)
            f_writer.writerow(['state', 'action_reward', 'n_state'])
            for user_histo in histo_to_write:
                states, actions = self.sample_histo(user_histo, action_ratio, 
                                                    max_samp_by_user, max_state, max_action, nb_states, nb_actions)
                for i in range(len(states)):
                    # FORMAT STATE
                    state_str = '|'.join(states[i])
                    # FORMAT ACTION
                    action_str = '|'.join(actions[i])
                    # FORMAT N_STATE
                    n_state_str = state_str + '|' + action_str
                    f_writer.writerow([state_str, action_str, n_state_str])


### Data

In [3]:
datapath = 'Books.csv'

In [4]:
# Hyperparameters
history_length = 12 # N in article
ra_length = 4 # K in article
discount_factor = 0.99 # Gamma in Bellman equation
actor_lr = 0.0001
critic_lr = 0.001
tau = 0.001 # τ in Algorithm 3
batch_size = 64
nb_episodes = 100
nb_rounds = 50
filename_summary = 'summary.txt'
alpha = 0.5 # α (alpha) in Equation (1)
gamma = 0.9 # Γ (Gamma) in Equation (4)
buffer_size = 1000000 # Size of replay memory D in article
fixed_length = True # Fixed memory length


dg = DataGenerator(datapath)
dg.gen_train_test(0.8, seed=42)

print(len(dg.train))
print(len(dg.test))
print('train: ', dg.train[:10])
print('test:', dg.test[:10])

#dg.write_csv('books_train.csv', dg.train, nb_states=[history_length], nb_actions=[ra_length])
#dg.write_csv('books_test.csv', dg_test, nb_states=[history_length], nb_actions=[ra_length])

#data = read_file('books_train.csv')

           item            user  rating   timestamp
0    0001713353  A1C6M8LCIX4M6M     5.0  1123804800
1    0001713353  A1REUF3A1YCPHM     5.0  1112140800
2    0001713353   A1YRBRK2XM5D5     5.0  1081036800
3    0001713353  A1V8ZR5P78P4ZU     5.0  1077321600
4    0001713353  A2ZB06582NXCIV     5.0  1475452800
..          ...             ...     ...         ...
995  0001384198  A2WDPKL01ILCQ5     3.0  1390089600
996  0001384198  A15WQ7V1OGJ2IE     5.0  1389744000
997  0001384198  A3O11AZC86MP9E     5.0  1389398400
998  0001384198   ARWYGWXB243RI     5.0  1389225600
999  0001384198  A331AFVDOICK1E     3.0  1389052800

[1000 rows x 4 columns]
['0001384198', '0001384198', '0001384198', '0002005263', '0001713353', '0001384198', '0001932349', '0001384198', '0002005263', '0001384198', '0001384198', '0002005263', '0001384198', '0001384198', '0001384198', '0001384198', '0001384198', '0001061240', '0001384198', '0002005263', '0001384198', '0001384198', '0001384198', '0001384198', '0001384198', 

## Embeddings Generator
---

In [5]:
int('003')

3

In [6]:
class EmbeddingsGenerator:
    def __init__(self, train_users, data):
        self.train_users = train_users
        
        # preprocess
        self.data = data.sort_values(by=['timestamp'])
        # make them start at 0
        # 유저아이디 인트로 인덱싱해줘야함
        
        user_uniq = self.data['user'].unique()
        user_ix = [num for num in range(len(user_uniq))]
        #user_ix_uniq = dict(zip(user_ix, user_uniq))
        user_uniq_ix = dict(zip(user_uniq, user_ix))
        self.data.replace({"user": user_uniq_ix})
    
        item_uniq = self.data['item'].unique()
        item_ix = [num for num in range(len(item_uniq))]
        #user_ix_uniq = dict(zip(user_ix, user_uniq))
        item_uniq_ix = dict(zip(item_uniq, item_ix))
        self.data.replace({"item": item_uniq_ix})
    
        self.data['user'] = self.data['user'] 
        self.data['item'] = self.data['item'] 
        self.user_count = self.data['user'].max() + 1
        self.book_count = self.data['item'].max() + 1
        # list of rated books by each user
        self.user_books = {}
        
        for user in range(self.user_count):
            self.user_books[user] = self.data[self.data.user == user]['item'].tolist()
        self.m = self.model()
    
    def model(self, hidden_layer_size=100):
        m = Sequential()
        m.add(Dense(hidden_layer_size, input_shape=(1, self.book_count)))
        m.add(Dropout(0.2))
        m.add(Dense(self.book_count, activation='softmax'))
        m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return m
    
    def generate_input(self, user):
        '''
        Returns a context and a target for the user
        
        context: user's history with one random book removed
        target: id of random removed book
        '''
        user_books_count = len(self.user_books[user])
        # picking random book
        random_index = np.random.randint(0, user_books_count - 1) # -1 avoids taking the las book
        # setting target
        target = np.zeros((1, self.book_count))
        target[0][self.user_books[user][random_index]] = 1
        # setting context
        context = np.zeros((1, self.book_count))
        context[0][self.user_books[user][:random_index] + self.user_books[user][random_index + 1:]] = 1
        return context, target
    
    def train(self, nb_epochs=300, batch_size=10000):
        '''
        Trains the model from train_users's history
        '''
        for i in range(nb_epochs):
            print('%d/%d' % (i+1, nb_epochs))
            batch = [self.generate_input(user=np.random.choic(self.train_users) - 1) for _ in range(batch_size)]
            X_train = np.array([b[0] for b in batch])
            y_train = np.array([b[0] for b in batch])
            self.m.fit(X_train, y_train, epochs=1, validation_split=0.5)
        
    def test(self, test_users, batch_size=10000):
        '''
        Returns [loss, accuracy] on the test set
        '''
        batch_test = [self.generate_input(user=np.random.choice(test_users) - 1) for _ in range(batch_size)]
        X_test = np.array([b[0] for b in batch_test])
        y_test = np.array([b[1] for b in batch_test])
        return self.m.evaluate(X_test, y_test)
    
    def save_embeddings(self, file_name):
        '''
        Generates a csv file containing the vecotr embedding for each book
        '''
        inp = self.m.input                                          # input placeholder
        outputs = [layer.output for layer in self.m.layers]         # all layer outputs
        functor = K.function([inp, K.learning_phase()])             # evaluation function
        
        # append embeddings to vectors
        vectors = []
        for book_id in range(self.book_count):
            book = np.zeros((1, 1, self.book_count))
            book[0][0][book_id] = 1
            layer_outs = fuctor([book])
            vector = [str(v) for v in layer_outs[0][0][0]]
            vector = '|'.join(vector)
            vectors.append([book_id, vector])
        
        # saves as a csv file
        embeddings = pd.DataFrame(vectors, columns=['item_id', 'vectors']).astype({'book_id': 'int32'})
        embeddings.to_csv(file_name, sep=';', index=False)
        files.download(file_name)

In [7]:
class Embeddings:
    def __init__(self, item_embeddings):
        self.item_embeddings = item_embeddings
    
    def size(self):
        return self.item_embeddings.shape[1]

    def get_embedding_vector(self):
        return self.item_embeddings
    
    def get_embedding(self, item_index):
        return self.item_embeddings[item_index]
    
    def embed(self, item_list):
        return np.array([self.get_embeddiing(item) for item in item_list])

In [8]:
def read_file(data_path):
    '''
    Load data from train.csv or test.csv
    '''
    data = pd.read_csv(data_path, sep=';')
    for col in ['state', 'n_state', 'action_reward']:
        data[col] = [np.array([[np.int(k) for k in ee.split('&')] for ee in e.split('|')]) for e in data[col]]
    for col in ['state', 'n_state']:
        data[col] = [np.array([e[0] for e in l]) for l in data[col]]
    
    data['action'] = [[e[0] for e in l] for l in data['action_reward']]
    data['reward'] = [tuple(e[1] for e in l) for l in data['action_reward']]
    data.drop(columns=['action_reward'], inplace=True)
    
    return data

def read_embeddings(embeddings_path):
    '''
    Load embeddings (a vector for each item)
    '''
    embeddings = pd.read_csv(embeddings_path, sep=';')
    
    return np.array([[np.float64(k) for k in e.split('|')] for e in embeddings['vectors']])
    

### Embeddings

In [ ]:
eg = EmbeddingsGenerator(dg.user_train, pd.read_csv('Books.csv', sep='\t', names=['user', 'item', 'rating', 'timestamp']))
eg.train(np_epochs=300)
train_loss, train_accuracy = eg.test(df.user_train)
print('Train set: Loss=%.4f ; Accuracy=%.1f%%' % (train_loss, train_accuracy * 100))
test_loss, test_accuracy = eg.test(dg.user_test)
print('Test set; Loss=%.4f; Accuracy=%.1f%%' % (test_loss, test_accuracy * 100))
eg.save_embeddings('embeddings.csv')
